## Uncorrected w/ ICV

In [5]:
import os
import os.path as op
import json
import shutil
import tarfile
import subprocess as sp
from glob import glob
import pandas as pd
import numpy as np
import re

pattern = re.compile("Estimated Total Intracranial Volume, (\d+\.\d+)", re.MULTILINE) #re pattern for RE
hp_SFvols = {"Participant":[], "ICV":[], "lh_CA1":[], "lh_CA3+4":[], "lh_subiculum":[],
                                         "rh_CA1":[], "rh_CA3+4":[], "rh_subiculum":[]}
base_dir = "/home/data/madlab/Reeb_EBC/derivatives/freesurfer6/"
base_glob = sorted(glob(op.join(base_dir, "*"))) #all folders in base_dir
subjects = [x[-7:] for x in base_glob if "sub-" in x] #obtain ids for all contained subject folders
for sub in subjects: 
    lh_vols_file = op.join(base_dir, "{0}/mri/lh.hippoSfVolumes-T1.v10.txt".format(sub)) 
    rh_vols_file = op.join(base_dir, "{0}/mri/rh.hippoSfVolumes-T1.v10.txt".format(sub)) 
    with open(op.join(base_dir, "{0}/stats/aseg.stats".format(sub)), "r") as aseg_stats: #read aseg.stats file
        lines = ""
        for line in aseg_stats: #iterate through lines in txt file
            lines += line #concatenate into a single variable
    icv = [x for x in re.finditer(pattern, lines)][0].group(1) #isolate total intracranial volume in lines
    #open hemisphere txt, load as dataframe, transpose, and assign subfield label as column header  
    hemis = {}
    hemis["lh"] = pd.read_table(lh_vols_file, header = None, delimiter = " ", index_col = 0).T 
    hemis["rh"] = pd.read_table(rh_vols_file, header = None, delimiter = " ", index_col = 0).T
        
    hp_SFvols["Participant"].append(sub) #preserve participant id
    hp_SFvols["ICV"].append(icv)
    #sum bihemispheric subfield volume, correct by dividing sum by total intracranial volume from aseg.stats
    for key in hemis:
        curr_hemi = hemis[key]
        hp_SFvols["{0}_CA1".format(key)].append(curr_hemi["CA1"].values[0])
        #CA3, CA4, molecular_layer_HP, and GC-ML-DG
        hp_SFvols["{0}_CA3+4".format(key)].append(curr_hemi["CA3"].values[0]+ 
                                                  curr_hemi["CA4"].values[0] + 
                                                  curr_hemi["molecular_layer_HP"].values[0] + 
                                                  curr_hemi["GC-ML-DG"].values[0])
        #subiculum, presubiculum, and parasubiculum
        hp_SFvols["{0}_subiculum".format(key)].append(curr_hemi["subiculum"].values[0] + 
                                                      curr_hemi["presubiculum"].values[0] + 
                                                      curr_hemi["parasubiculum"].values[0])

hp_SFvols = pd.DataFrame(hp_SFvols) #convert dict to df
hp_SFvols = hp_SFvols.set_index("Participant") #set participant as row keys
hp_SFvols.to_csv(base_dir + "hp_subfieldvols_hemiscorrected.csv") #save df to csv

In [60]:
lh_vols

,Hippocampal_tail,subiculum,CA1,hippocampal-fissure,presubiculum,parasubiculum,molecular_layer_HP,GC-ML-DG,CA3,CA4,fimbria,HATA,Whole_hippocampus
1,413.992033,341.725287,505.78729,135.111046,262.852746,59.68436,460.599174,238.463862,188.258924,207.467717,47.598007,46.222088,2772.651486


In [119]:
hp_SFvols

,Participant,CA1,CA3+4,subiculum
0,sub-133,0.000775,0.001570,0.001101
1,sub-131,0.000746,0.001509,0.001051
2,sub-104,0.000761,0.001398,0.000971
3,sub-102,0.000839,0.001679,0.000987
4,sub-125,0.000844,0.001618,0.001116
5,sub-123,0.000749,0.001693,0.001053
6,sub-121,0.000861,0.001623,0.001069
7,sub-117,0.000759,0.001604,0.001013
8,sub-115,0.000727,0.001546,0.000996
9,sub-134,0.000859,0.001790,0.001053


In [117]:
#corrected
hp_SFvols

,Participant,CA1,CA3+4,subiculum
0,sub-101,0.000793,0.001665,0.000979


In [111]:
#uncorrected
hp_SFvols

,Participant,CA1,CA3+4,subiculum
0,sub-101,1043.027424,2189.258299,1287.942549


## Uncorrected

In [3]:
import os
import os.path as op
import json
import shutil
import tarfile
import subprocess as sp
from glob import glob
import pandas as pd
import numpy as np
import re

pattern = re.compile("Estimated Total Intracranial Volume, (\d+\.\d+)", re.MULTILINE) #pattern for RE
hp_SFvols = {"Participant":[], "CA1":[], "CA3+4":[], "subiculum":[], "ICV":[]} #corrected hippocampal subfield volumes
base_dir = "/home/data/madlab/Reeb_EBC/derivatives/freesurfer6/"
base_glob = sorted(glob(op.join(base_dir, "*"))) #all folders in base_dir
subjects = [x[-7:] for x in base_glob if "sub-" in x] #obtain ids for all contained subject folders
for sub in subjects: 
    lh_vols_file = op.join(base_dir, "{0}/mri/lh.hippoSfVolumes-T1.v10.txt".format(sub)) 
    rh_vols_file = op.join(base_dir, "{0}/mri/rh.hippoSfVolumes-T1.v10.txt".format(sub)) 
    with open(op.join(base_dir, "{0}/stats/aseg.stats".format(sub)), "r") as aseg_stats: #read aseg.stats file
        lines = ""
        for line in aseg_stats: #iterate through lines in txt file
            lines += line #concatenate into a single variable
    icv = [x for x in re.finditer(pattern, lines)][0].group(1) #isolate total intracranial volume in lines
    #open hemisphere txt, load as dataframe, transpose, and assign subfield label as column header  
    lh_vols = pd.read_table(lh_vols_file, header = None, delimiter = " ", index_col = 0).T 
    rh_vols = pd.read_table(rh_vols_file, header = None, delimiter = " ", index_col = 0).T
    
    hp_SFvols["Participant"].append(sub) #preserve participant id
    #sum bihemispheric subfield volume, correct by dividing sum by total intracranial volume from aseg.stats
    hp_SFvols["CA1"].append(lh_vols["CA1"].values[0] + rh_vols["CA1"].values[0])
    #CA3, CA4, molecular_layer_HP, and GC-ML-DG
    hp_SFvols["CA3+4"].append(lh_vols["CA3"].values[0] + rh_vols["CA3"].values[0] +
                              lh_vols["CA4"].values[0] + rh_vols["CA4"].values[0] +
                              lh_vols["molecular_layer_HP"].values[0] + rh_vols["molecular_layer_HP"].values[0] +
                              lh_vols["GC-ML-DG"].values[0] + rh_vols["GC-ML-DG"].values[0])
    #subiculum, presubiculum, and parasubiculum
    hp_SFvols["subiculum"].append(lh_vols["subiculum"].values[0] + rh_vols["subiculum"].values[0] +
                                  lh_vols["presubiculum"].values[0] + rh_vols["presubiculum"].values[0] +
                                  lh_vols["parasubiculum"].values[0] + rh_vols["parasubiculum"].values[0])
    hp_SFvols["ICV"].append(icv)

hp_SFvols = pd.DataFrame(hp_SFvols) #convert dict to df
hp_SFvols = hp_SFvols.set_index("Participant") #set participant as row keys
hp_SFvols.to_csv(base_dir + "hp_subfieldvols_uncorrected.csv") #save df to csv